How Python evaluates decorator syntax

How Python decides whether a variable is local

Why closures exist and how they work

What problem is solved by nonlocal

In [1]:
def deco(func):
    def inner():
        print("run inner")
    return inner

@deco
def target():
    print("run target")

target()

run inner


In [ ]:
# decorator is applied to the function target, so when we call target(), it will actually call the inner function returned by deco, not the original
target

<function __main__.deco.<locals>.inner()>

When Python Executes Decorators

A key feature of decorators is that they run right after the decorated function is defined. same as import module

In [4]:
registry = []

def register(func):
    print("registering", func.__name__)
    registry.append(func)
    return func

@register
def f1():
    print("f1")

@register
def f2():
    print("f2")

def f3():
    print("f3")
    
def run():
    print("running")
    print("registry:", registry)
    f1()
    f2()
    f3()

run()


registering f1
registering f2
running
registry: [<function f1 at 0x000001A8EA0B1E40>, <function f2 at 0x000001A8EA0B20C0>]
f1
f2
f3


Variable Scope Rules

In [5]:
def f1(a):
    print("f1", a)
    print("f1", b)
f1(3)

f1 3


NameError: name 'b' is not defined

In [6]:
b=6
f1(3)

f1 3
f1 6


In [ ]:
b=6
def f2(a):
    print("f2", a)
    print("f2", b)
    b = 9 #cannot access local variable 'b' where it is not associated with a value
f2(3)

f2 3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [9]:
b=6
def f3(a):
    global b
    print("f3", a)
    print("f3", b)
    b = 9
f3(3)
b

f3 3
f3 6


9

The module global scope
Made of names assigned to values outside of any class or function block.

The f3 function local scope
Made of names assigned to values as parameters, or directly in the body of the function.

In [ ]:
from dis import dis
dis(f1)
# 38 -> global b

  1           0 RESUME                   0

  2           2 LOAD_GLOBAL              1 (NULL + print)
             12 LOAD_CONST               1 ('f1')
             14 LOAD_FAST                0 (a)
             16 CALL                     2
             24 POP_TOP

  3          26 LOAD_GLOBAL              1 (NULL + print)
             36 LOAD_CONST               1 ('f1')
             38 LOAD_GLOBAL              2 (b)
             48 CALL                     2
             56 POP_TOP
             58 RETURN_CONST             0 (None)


In [ ]:
dis(f2)
# 38 -> local b

  2           0 RESUME                   0

  3           2 LOAD_GLOBAL              1 (NULL + print)
             12 LOAD_CONST               1 ('f2')
             14 LOAD_FAST                0 (a)
             16 CALL                     2
             24 POP_TOP

  4          26 LOAD_GLOBAL              1 (NULL + print)
             36 LOAD_CONST               1 ('f2')
             38 LOAD_FAST_CHECK          1 (b)
             40 CALL                     2
             48 POP_TOP

  5          50 LOAD_CONST               2 (9)
             52 STORE_FAST               1 (b)
             54 RETURN_CONST             0 (None)


Actually, a closure is a function—let’s call it f—with an extended scope that encompasses variables referenced in the body of f that are not global variables or local variables of f. Such variables must come from the local scope of an outer function that encompasses f.

闭包：内部函数使用外部函数的free variables

In [12]:
class Average:
    def __init__(self):
        self.series = []
    
    def __call__(self, value):
        self.series.append(value)
        return sum(self.series) / len(self.series)
    
avg = Average()
print(avg(10))
print(avg(20))
print(avg(30))

10.0
15.0
20.0


In [13]:
def avg():
    series = []
    def inner(value):
        series.append(value)
        return sum(series) / len(series)
    return inner

avg = avg()
print(avg(10))
print(avg(20))
print(avg(30))

10.0
15.0
20.0


In [14]:
avg.__code__.co_varnames

('value',)

free variables:

In [15]:
avg.__code__.co_freevars

('series',)

In [16]:
avg.__closure__

(<cell at 0x000001A8EA123EB0: list object at 0x000001A8EA739DC0>,)

In [17]:
avg.__closure__[0].cell_contents

[10, 20, 30]

free variable 存在closure里，closure和avg绑定

The value for series is kept in the __closure__ attribute of the returned function avg. Each item in avg.__closure__ corresponds to a name in avg.​__code__​.co_freevars. These items are cells, and they have an attribute called cell_contents where the actual value can be found. Example 9-11 shows these attributes.

To summarize: a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

In [18]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

In [19]:
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [20]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total  # use nonlocal to modify the outer scope variables
        count += 1
        total += new_value
        return total / count

    return averager

In [21]:
avg = make_averager()
avg(10)

10.0

Variable Lookup Logic
When a function is defined, the Python bytecode compiler determines how to fetch a variable x that appears in it, based on these rules:3

If there is a global x declaration, x comes from and is assigned to the x global variable module.4

If there is a nonlocal x declaration, x comes from and is assigned to the x local variable of the nearest surrounding function where x is defined.

If x is a parameter or is assigned a value in the function body, then x is the local variable.

If x is referenced but is not assigned and is not a parameter:

x will be looked up in the local scopes of the surrounding function bodies (nonlocal scopes).

If not found in surrounding scopes, it will be read from the module global scope.

If not found in the global scope, it will be read from __builtins__.__dict__.

variable 查找过程： global -> nonlocal -> parameter -> local scope

Implementing a Simple Decorator

In [22]:
import time


def clock(func):
    def clocked(*args):  
        t0 = time.perf_counter()
        result = func(*args)  
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked  

In [23]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12322190s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000040s] factorial(1) -> 1
[0.00001240s] factorial(2) -> 2
[0.00001960s] factorial(3) -> 6
[0.00002660s] factorial(4) -> 24
[0.00003360s] factorial(5) -> 120
[0.00004120s] factorial(6) -> 720
6! = 720


In [24]:
def test_deco(func):
    def inner(*args):
        print("run inner")
        return func(*args)
    return inner

@test_deco
def test_add(a, b):
    return a + b

print(test_add(1, 2))

run inner
3


In [25]:
test_add.__name__

'inner'

In [ ]:
import functools


def test_deco(func):
    @functools.wraps(func)  # use functools.wraps to preserve the original function's metadata
    def inner(*args):
        print("run inner")
        return func(*args)
    return inner

@test_deco
def test_add(a, b):
    return a + b

print(test_add(1, 2))

run inner
3


In [27]:
test_add.__name__

'test_add'

@functools.wraps(func) # copy the metadata from func to clocked

In [28]:
import time
import functools


def clock(func):
    @functools.wraps(func) # copy the metadata from func to clocked
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked